In [16]:
import requests
import pandas as pd
from datetime import datetime
from datetime import timedelta
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [43]:
link = 'https://thanhnien.vn/tim-kiem/Y8O6bSBnaWEgY-G6p20=/cum-gia-cam.html?m=0&t=&c=0&y=-1&o='
root = 'https://thanhnien.vn'
df = pd.DataFrame(columns = ['Title', 'URL', 'Date', 'Content'])

In [44]:
def scrape_thanhnien(link, cur_link, root, df, start=0, end=15):
    if start == end:
        return df
    html = requests.get(cur_link).text
    soup = BeautifulSoup(html, 'lxml')
    news = soup.find_all('article', class_ = 'story')
    for item in news:
        title = item.find('a', class_ = 'story__thumb')['title']
        sub_url = item.find('a', class_ = 'story__thumb')['href']
        date = datetime.strptime(item.find('time', class_ = 'timebox').text, '%H:%M, %d/%m/%Y')
        if sub_url.startswith('https'):
            url = sub_url
            response = requests.get(url).text
        else:
            url = root + sub_url
            response = requests.get(url).text
        new_soup = BeautifulSoup(response, 'lxml')
        content = new_soup.find('div', class_ = 'cms-body detail').text
        new_row = {'Title': title, 'URL': url, 'Date': date, 'Content': content}
        df = df.append(new_row, ignore_index = True)
    start += 1
    next = link + '&p=' + str(start)
    return scrape_thanhnien(link, next, root, df, start, end)

In [45]:
df = scrape_thanhnien(link, link, root, df)

In [46]:
df

,Title,URL,Date,Content
0,Trung Quốc ghi nhận ca cúm gia cầm H5N6 ở người,https://thanhnien.vn/the-gioi/trung-quoc-ghi-n...,2021-07-16 07:00:00,"\nBệnh nhân là người đàn ông 55 tuổi, đã được ..."
1,"3 tỉnh có cúm gia cầm A/H5N8 thể độc lực cao, ...",https://thanhnien.vn/tai-chinh-kinh-doanh/3-ti...,2021-07-06 15:21:00,"\nNgày 6.7, thông tin từ Bộ NN-PTNT cho biết, ..."
2,Cúm gia cầm H5N8 sẽ là đại dịch kế tiếp?,https://thanhnien.vn/video/the-gioi/cum-gia-ca...,2021-06-27 08:00:00,\nCác chuyên gia đã viết một bài báo trên tạp ...
3,Nga ghi nhận trường hợp người nhiễm cúm gia cầ...,https://thanhnien.vn/the-gioi/nga-ghi-nhan-tru...,2021-02-21 09:14:00,\nĐợt bùng phát virus H5N8 đã được ghi nhận ở ...
4,Cà Mau: Phát hiện ổ cúm gia cầm H5N1 ở H.Trần ...,https://thanhnien.vn/thoi-su/ca-mau-phat-hien-...,2020-09-07 17:06:00,"\nNgày 7.9, tin từ UBND xã Khánh Bình Đông, H...."
...,...,...,...,...
295,Nhật Bản: Lập kho dự trữ vắc-xin phòng cúm gia...,https://thanhnien.vn/the-gioi/nhat-ban-lap-kho...,2007-01-04 16:08:00,\nChính phủ Nhật Bản sẽ xem xét thông qua kế h...
296,Kiểm tra công tác phòng chống dịch cúm gia cầm...,https://thanhnien.vn/thoi-su/kiem-tra-cong-tac...,2007-01-04 00:24:00,"\nCùng ngày, Cục Thú y cho biết, hiện nay dịc..."
297,T.Ư Đoàn: Khẩn trương tổ chức lực lượng thanh ...,https://thanhnien.vn/gioi-tre/tu-doan-khan-tru...,2007-01-03 22:12:00,"\nTheo đó, các tỉnh, thành đoàn triển khai nga..."
298,Thêm 5 tỉnh nằm trong vùng nguy hiểm của dịch ...,https://thanhnien.vn/thoi-su/them-5-tinh-nam-t...,2007-01-02 23:24:00,"\nĐó là các tỉnh Quảng Nam, Quảng Ngãi, Bắc Gi..."


In [47]:
print(df['Date'].min(), df['Date'].max())

2007-01-01 23:49:00 2021-07-16 07:00:00


In [48]:
df.to_csv('data/thanhnien_articles_vietnamese.csv', index = False)